In [28]:
import pandas as pd
import numpy as np
import ast
from sklearn.impute import SimpleImputer

In [29]:
job_df = pd.read_csv('job_cleaned.csv')
job_df['programming language'] = job_df['programming language'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)
job_df = job_df[job_df['programming language'].str.len() > 0].reset_index(drop=True)
job_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2717 entries, 0 to 2716
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   job title             2717 non-null   object 
 1   company name          2717 non-null   object 
 2   location              2717 non-null   object 
 3   salary                2717 non-null   float64
 4   job type              2317 non-null   object 
 5   job description       2717 non-null   object 
 6   programming language  2717 non-null   object 
dtypes: float64(1), object(6)
memory usage: 148.7+ KB


In [30]:
d = {}

def note_numlanguage(languages):
    for lang_list in languages:
        for lang in lang_list:  
            if lang in d:
                d[lang] += 1  
            else:
                d[lang] = 1  

note_numlanguage(job_df['programming language'])

language_df = pd.DataFrame(columns=['programming language', 'job amount'])
print(len(d))
for key, value in d.items():
    new_row ={'programming language':key, 'job amount':value}
    language_df = language_df._append(new_row, ignore_index=True)
    print(f"{key}: {value}")

175
Dart: 32
Java: 728
PHP: 505
JavaScript: 1111
PostgreSQL: 365
SQL: 793
MySQL: 511
TypeScript: 211
Visual Basic: 234
Z++: 99
Flow: 120
CHAIN: 64
FOCUS: 326
Python: 675
kernel: 1
Address: 141
C: 362
JS++: 181
C#: 225
Go: 211
Ruby: 112
Ada: 71
GraphQL: 62
MariaDB: 24
Scheme: 6
Copilot: 8
Expect: 33
Solidity: 4
solid: 338
pipelines: 244
RAPID: 31
NPL: 1
april: 3
flagship: 11
Kotlin: 117
Swift: 96
Small: 72
Simplicity: 5
Perl: 18
C++: 118
R: 148
Scratch: 25
F: 14
Scala: 36
Rust: 27
Groovy: 10
Revolution: 61
MATLAB: 17
Adept: 21
Click: 54
Apex: 11
.QL: 26
dss: 2
Pure: 3
SPARK: 82
GAP: 23
Alpha: 3
Falcon: 2
Plot: 46
Joy: 57
CONVERT: 23
Frank: 1
P: 38
lever: 1
JADE: 5
Blue: 3
THINK C: 11
Inform: 32
NATURAL: 68
Reason: 11
ARM: 6
SR: 6
Crystal: 11
PowerHouse: 11
Conceptual: 22
Factor: 10
False: 2
RPL: 4
SQLite: 12
fe: 7
Chrome: 3
Tao: 4
Flavors: 4
Cyber: 15
Bash: 42
DEMOS: 7
Q: 4
PROMETHEUS: 29
Gist: 3
PILOT: 6
REDUCE: 28
Monkey: 1
Z: 1
abs: 1
FaCT: 5
MACRO: 2
FLUX: 4
ECMAScript: 3
Argus: 3
A

In [31]:
language_list = language_df['programming language'].to_list()
language_salary_dict={}
for lang in language_list:
    language_salary_dict[lang] = 0
print(language_salary_dict)

{'Dart': 0, 'Java': 0, 'PHP': 0, 'JavaScript': 0, 'PostgreSQL': 0, 'SQL': 0, 'MySQL': 0, 'TypeScript': 0, 'Visual Basic': 0, 'Z++': 0, 'Flow': 0, 'CHAIN': 0, 'FOCUS': 0, 'Python': 0, 'kernel': 0, 'Address': 0, 'C': 0, 'JS++': 0, 'C#': 0, 'Go': 0, 'Ruby': 0, 'Ada': 0, 'GraphQL': 0, 'MariaDB': 0, 'Scheme': 0, 'Copilot': 0, 'Expect': 0, 'Solidity': 0, 'solid': 0, 'pipelines': 0, 'RAPID': 0, 'NPL': 0, 'april': 0, 'flagship': 0, 'Kotlin': 0, 'Swift': 0, 'Small': 0, 'Simplicity': 0, 'Perl': 0, 'C++': 0, 'R': 0, 'Scratch': 0, 'F': 0, 'Scala': 0, 'Rust': 0, 'Groovy': 0, 'Revolution': 0, 'MATLAB': 0, 'Adept': 0, 'Click': 0, 'Apex': 0, '.QL': 0, 'dss': 0, 'Pure': 0, 'SPARK': 0, 'GAP': 0, 'Alpha': 0, 'Falcon': 0, 'Plot': 0, 'Joy': 0, 'CONVERT': 0, 'Frank': 0, 'P': 0, 'lever': 0, 'JADE': 0, 'Blue': 0, 'THINK C': 0, 'Inform': 0, 'NATURAL': 0, 'Reason': 0, 'ARM': 0, 'SR': 0, 'Crystal': 0, 'PowerHouse': 0, 'Conceptual': 0, 'Factor': 0, 'False': 0, 'RPL': 0, 'SQLite': 0, 'fe': 0, 'Chrome': 0, 'Tao': 0

In [32]:
for _, row in job_df.iterrows():
    salary = row['salary']
    for lang in row['programming language']:
        language_salary_dict[lang] += salary

print(language_salary_dict)

{'Dart': 200005138.40966603, 'Java': 4479768447.639802, 'PHP': 2909950931.938514, 'JavaScript': 6852822430.894201, 'PostgreSQL': 1839993798.0923576, 'SQL': 5360096604.482176, 'MySQL': 2699152586.2269125, 'TypeScript': 1395920198.2517376, 'Visual Basic': 1671768775.090993, 'Z++': 1172856090.8839326, 'Flow': 1114883780.38426, 'CHAIN': 972304055.6073251, 'FOCUS': 2690734522.305558, 'Python': 4272074951.70023, 'kernel': 34507957.0, 'Address': 888272557.5543638, 'C': 2146290239.1312919, 'JS++': 1013739970.8601706, 'C#': 1413156668.8828928, 'Go': 1655744374.0390837, 'Ruby': 823410051.0168891, 'Ada': 546793184.6739922, 'GraphQL': 356615000.12044597, 'MariaDB': 123791140.91538097, 'Scheme': 71066848.04069766, 'Copilot': 83329155.05711356, 'Expect': 591377169.8926609, 'Solidity': 38084271.47797595, 'solid': 2157237398.4837484, 'pipelines': 1356734670.2622058, 'RAPID': 170448582.83980486, 'NPL': 11004708.625, 'april': 14125541.958333334, 'flagship': 68237628.49458727, 'Kotlin': 713819916.3668859

In [33]:
amount_list = language_df['job amount'].to_list()

i=0
for key,val in language_salary_dict.items():
    language_salary_dict[key] = val/amount_list[i]
    i+=1

print(language_salary_dict)


{'Dart': 6250160.5753020635, 'Java': 6153528.08741731, 'PHP': 5762279.073145573, 'JavaScript': 6168157.003505131, 'PostgreSQL': 5041078.898883171, 'SQL': 6759264.31838862, 'MySQL': 5282098.994573214, 'TypeScript': 6615735.536738093, 'Visual Basic': 7144311.004662363, 'Z++': 11847031.221049825, 'Flow': 9290698.169868832, 'CHAIN': 15192250.868864454, 'FOCUS': 8253786.878237909, 'Python': 6328999.928444785, 'kernel': 34507957.0, 'Address': 6299805.372725985, 'C': 5928978.561136166, 'JS++': 5600773.319669451, 'C#': 6280696.3061461905, 'Go': 7847129.7347823875, 'Ruby': 7351875.455507939, 'Ada': 7701312.4601970725, 'GraphQL': 5751854.840652354, 'MariaDB': 5157964.20480754, 'Scheme': 11844474.673449611, 'Copilot': 10416144.382139195, 'Expect': 17920520.2997776, 'Solidity': 9521067.869493987, 'solid': 6382359.167111682, 'pipelines': 5560387.992877893, 'RAPID': 5498341.381929189, 'NPL': 11004708.625, 'april': 4708513.986111111, 'flagship': 6203420.772235206, 'Kotlin': 6101024.9262127, 'Swift': 

In [34]:
language_salary_df = pd.DataFrame(list(language_salary_dict.items()), columns=['programming language', 'average salary'])
language_df = pd.merge(language_df, language_salary_df, on='programming language')

language_df.head()

,programming language,job amount,average salary
0,Dart,32,6.250161e+06
1,Java,728,6.153528e+06
2,PHP,505,5.762279e+06
3,JavaScript,1111,6.168157e+06
4,PostgreSQL,365,5.041079e+06


In [35]:
language_df.to_csv('language_list.csv', index=False)

In [36]:
language_df.describe()

,average salary
count,1.750000e+02
mean,8.259396e+06
std,9.028563e+06
min,7.763158e+05
25%,4.412282e+06
50%,6.329000e+06
75%,9.448299e+06
max,8.588051e+07
